In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from numpy import sin, cos, sqrt, exp
import yfinance as yf
from datetime import datetime, timedelta
import json

def trigfit(x, n, w, m, a, b, xm):
    Sc =0.0
    Ss =0.0
    Scc=0.0
    Sss=0.0
    Scs=0.0
    Sx =0.0
    Sxc=0.0
    Sxs=0.0
    for i in range(n):
        c = cos(w*i)
        s = sin(w*i)
        dx = x[i] - xm[i]
        Sc +=c
        Ss +=s
        Scc+=c*c
        Sss+=s*s
        Scs+=c*s
        Sx +=dx
        Sxc+=dx*c
        Sxs+=dx*s
    Sc /=n
    Ss /=n
    Scc/=n
    Sss/=n
    Scs/=n
    Sx /=n
    Sxc/=n
    Sxs/=n   
    if w == 0 or None:
        m = Sx
        a = 0.0
        b = 0.0
    else:
        den=(Scs-Sc*Ss)**2-(Scc-Sc*Sc)*(Sss-Ss*Ss)
        a=((Sxs-Sx*Ss)*(Scs-Sc*Ss)-(Sxc-Sx*Sc)*(Sss-Ss*Ss))/den
        b=((Sxc-Sx*Sc)*(Scs-Sc*Ss)-(Sxs-Sx*Ss)*(Scc-Sc*Sc))/den
        m=Sx-a*Sc-b*Ss
    return w, m, a ,b 
def freq(x, n, w, m, a, b, xm):
    FreqTOL =0.00001
    z = [None] * n
    alpha = 0.0 # = beta for initialization
    beta = 2.0
    z[0] = x[0] - xm[0]
    count = 0
    while abs(alpha - beta) > FreqTOL:
        count += 1
        alpha = beta
        z[1] = x[1] = xm[1] + alpha * z[0]
        num = z[0] * z[1]
        den = z[0] * z[0]
        for i in range(2, n):
            z[i] = x[i] - xm[i] + alpha*z[i-1] - z[i-2]
            num += z[i-1]*(z[i]+z[i-2])
            den+=z[i-1]*z[i-1]
        beta = num/den
        if count >= 1000:
            print("break")
            break
    if -1 <= beta/2.0 <= 1:
        w = np.arccos(beta/2.0)
    else:
        w= 0 
    w, m, a, b = trigfit(x, n, w, m, a, b, xm)
    return w, m, a, b


def main(df_close_unpivoted, ticker):
    shop = df_close_unpivoted[ticker]
    av = 0.0

    past = 252
    pred = 100
    # past closed price
    x = [None] * past
    close = shop
    close = close[::-1]
    # average price for the past 300(past) days
    xm = [None] * past
    ym = [None] * 101
    for i in range(past):
        x[i] = close[i]
        av += x[i]
    av/=past

    for i in range(past):
        xm[i] = av
        if(i <= pred):
            ym[i] = av
    harmonics = 20
    w = 0.0
    m = 0.0
    a = 0.0
    b = 0.0
    for i in range(1, harmonics + 1):
        w, m, a, b = freq(x, past, w, m, a, b, xm)
        for i in range(past):
            xm[i] += m+a*cos(w*i) + b*sin(w*i)
            if(i <= pred):
                ym[i] += m+a*cos(w*i)-b*sin(w*i)
    return ym

def getNavSeries(df_now_for_nav, weighting=[0.2] * 5, nav_current = 1):
    df2 = (1 + df_now_for_nav.pct_change()).cumprod()
    df3 = df2.fillna(1)
    df4 = df3*nav_current
    nav_series = np.sum(df4 * weighting, axis=1)
    return nav_series


In [3]:
import pandas as pd
df = pd.read_csv("../factor/close_price_top290.csv")
alls = list(df.columns[200:250])
symbols = [i.split(".")[0] for i in alls]

In [17]:
df_close_unpivoted = yf.download(alls, start='2022-01-01')['Adj Close']
df_close_unpivoted.dropna(how='any', axis=1, inplace=True)

[*********************100%%**********************]  50 of 50 completed

2 Failed downloads:
['1710.TW']: ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out. (read timeout=10)"))
['1517.TW']: ConnectionError(ReadTimeoutError("HTTPSConnectionPool(host='query2.finance.yahoo.com', port=443): Read timed out."))


In [20]:
# factor data are aligned with the date of calculation, all factors needs to be shift 1 for actual usage
start = "2023-02-01"
end = datetime.now()
factor_lists = []
for end_date in list(df_close_unpivoted[(df_close_unpivoted.index >= start) & (df_close_unpivoted.index <= end)].index):
    print(end_date)
    row_elements = []
    for ticker in list(df_close_unpivoted.columns):
        ym = main(df_close_unpivoted=df_close_unpivoted[df_close_unpivoted.index <= end_date], ticker=ticker)
        r = np.polyfit(np.arange(0, len(ym), 1), ym, 1)[0]
        row_elements.append(r) # all tickers for each day
    factor_lists.append(row_elements)

2023-02-01 00:00:00


/var/folders/zc/_sy5126x4ks5w74_nsrx_mv40000gp/T/ipykernel_13675/1094926580.py:91: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x[i] = close[i]


2023-02-02 00:00:00
2023-02-03 00:00:00
2023-02-06 00:00:00
2023-02-07 00:00:00
2023-02-08 00:00:00
2023-02-09 00:00:00
2023-02-10 00:00:00
2023-02-13 00:00:00
2023-02-14 00:00:00
2023-02-15 00:00:00
2023-02-16 00:00:00
2023-02-17 00:00:00
2023-02-20 00:00:00
2023-02-21 00:00:00
2023-02-22 00:00:00
2023-02-23 00:00:00
2023-02-24 00:00:00
2023-03-01 00:00:00
2023-03-02 00:00:00
2023-03-03 00:00:00
2023-03-06 00:00:00
2023-03-07 00:00:00
2023-03-08 00:00:00
2023-03-09 00:00:00
2023-03-10 00:00:00
2023-03-13 00:00:00
2023-03-14 00:00:00
2023-03-15 00:00:00
2023-03-16 00:00:00
2023-03-17 00:00:00
2023-03-20 00:00:00
2023-03-21 00:00:00
2023-03-22 00:00:00
2023-03-23 00:00:00
2023-03-24 00:00:00
2023-03-27 00:00:00
2023-03-28 00:00:00
2023-03-29 00:00:00
2023-03-30 00:00:00
2023-03-31 00:00:00
2023-04-06 00:00:00
2023-04-07 00:00:00
2023-04-10 00:00:00
2023-04-11 00:00:00
2023-04-12 00:00:00
2023-04-13 00:00:00
2023-04-14 00:00:00
2023-04-17 00:00:00
2023-04-18 00:00:00
2023-04-19 00:00:00


In [21]:

a = pd.DataFrame(factor_lists)
a.columns = df_close_unpivoted.columns
a.index = df_close_unpivoted[(df_close_unpivoted.index >= start) & (df_close_unpivoted.index <= end)].index
a.head()

Ticker,1475.TW,1476.TW,1477.TW,1503.TW,1512.TW,1513.TW,1514.TW,1516.TW,1525.TW,1526.TW,...,1617.TW,1618.TW,1626.TW,1701.TW,1709.TW,1713.TW,1721.TW,1723.TW,1732.TW,1734.TW
Date,,,,,,,,,,,,,,,,,,,,,
2023-02-01,-0.124173,-0.609184,-0.761906,-0.162920,-0.047486,-0.132793,-0.070070,-0.009539,-0.051326,0.120133,...,0.000233,-0.016312,0.005135,-0.034304,0.008907,-0.011229,0.047864,0.120392,0.126144,0.005204
2023-02-02,-0.131356,-0.636898,-0.764968,-0.168542,-0.043907,-0.111676,-0.073118,-0.010163,-0.051352,0.118647,...,0.013285,-0.016012,0.005296,-0.035948,0.012736,-0.011225,0.047500,0.120473,0.126239,0.005412
2023-02-03,-0.168191,-0.648573,-0.766276,-0.171496,-0.043772,-0.121908,-0.076609,-0.010539,-0.052232,0.116678,...,0.013086,-0.015844,0.005363,-0.049968,0.017787,-0.011219,0.054074,0.120554,0.129257,0.005555
2023-02-06,-0.168017,-0.655791,-0.768330,-0.176821,-0.044537,-0.121807,-0.079237,-0.011949,-0.053557,0.114004,...,0.012904,-0.015666,0.005457,-0.048941,0.010609,-0.011238,0.054159,0.120503,0.129945,0.005745
2023-02-07,-0.168124,-0.670903,-0.768444,-0.146877,-0.044097,-0.121519,-0.082380,-0.013488,-0.053793,0.111030,...,0.012719,-0.015576,0.005522,-0.046468,0.008576,-0.011121,0.054233,0.120245,0.131101,0.005715


In [10]:
all_regression_data = a.T.sort_values(by=0, ascending=False)
top10 = all_regression_data[:5]
top10_20230101 = list(top10.index)
top10 = top10.iloc[:, 0]

weighting_based_on_r = top10/sum(top10)
top10, weighting_based_on_r

(Ticker
 ULTA    1.618642
 PAYC    1.602173
 CHTR    1.310679
 BIO     1.142007
 ILMN    1.008613
 Name: 0, dtype: float64,
 Ticker
 ULTA    0.242235
 PAYC    0.239770
 CHTR    0.196147
 BIO     0.170905
 ILMN    0.150942
 Name: 0, dtype: float64)